# DATA 602 Assignment 12
# Dan Smilowitz

## Original Operation

In [1]:
%%timeit

import pandas as pd
import numpy as np
# read and sanitize data
aapl = pd.read_csv('data/apple.2011.csv', names=['Date', 'Price', 'Pct_Change'], header=0)
aapl['Date'] = pd.to_datetime(aapl['Date'], format='%m/%d/%Y')
aapl['Pct_Change'] = pd.to_numeric(aapl['Pct_Change'], errors='coerce')
# get mean & sd; create number generator
mu = aapl['Pct_Change'].mean()
sigma = aapl['Pct_Change'].std()
def aapl_gauss(n):
    return np.random.normal(mu, sigma, n)
# simulate 10000 times
sims = np.empty(10000)
for i in xrange(10000):
    price = aapl['Price'].iloc[-1]
    my_pct = aapl_gauss(20)
    for dy in xrange(20):
        price = price * (1 + my_pct[dy])
    sims[i] = price
# get Value at Risk
sims = sims.argsort()
VaR = np.percentile(sims, 1)

1 loop, best of 3: 291 ms per loop


## Parallel Processing

In [2]:
import ipyparallel
clients = ipyparallel.Client()
clients.block = True
dv = clients.direct_view()

In [3]:
%%timeit

# initiate arrays on clusters
%px from numpy import empty
%px sims = empty(2500)

%px

import numpy as np
import pandas as pd

aapl = pd.read_csv('data/apple.2011.csv', names=['Date', 'Price', 'Pct_Change'], header=0)
aapl['Date'] = pd.to_datetime(aapl['Date'], format='%m/%d/%Y')
aapl['Pct_Change'] = pd.to_numeric(aapl['Pct_Change'], errors='coerce')

mu = aapl['Pct_Change'].mean()
sigma = aapl['Pct_Change'].std()
def aapl_gauss(n):
    return np.random.normal(mu, sigma, n)

sims = np.empty(2500)
for i in xrange(2500):
    price = aapl['Price'].iloc[-1]
    my_pct = aapl_gauss(20)
    for dy in xrange(20):
        price = price * (1 + my_pct[dy])
    sims[i] = price

%px

# gather results from parallel clusters
sims = dv.gather('sims')
sims = np.array(sims)
sims = sims.argsort()
VaR = np.percentile(sims, 1)

1 loop, best of 3: 172 ms per loop


## Comparision
The time for the operation, as returned by the `%%timeit` magic command, shows that the parallel-processed solution executed over 40% faster than the serial-processed solution.  This does not account for the time to start and end the four clusters; this time leads to the total time of operation being longer for the parallel-processed solution in this case.  However, when working with larger datasets and more complex or operations, or in cases where the clusters are already running, the savings may be more significant.